# Ranking Measures Check

## Data Format:
### Golden results
Golden DataFrame Format is standardized to 2 columns: query, document:

```python
query	document
0	q1	doc2
1	q1	doc3
2	q2	doc6
```

### Search results
Search results can be given in 2 formats:

#### Flat format:
```python
	query	document	rank
0	q1	doc1	1
1	q1	doc2	2
2	q1	doc3	3
3	q2	doc4	1
4	q2	doc5	2
5	q2	doc6	3
```

#### Nested format:
```python
[{'question': 'q1', 'answers': ['doc1', 'doc2', 'doc3']},
 {'question': 'q2', 'answers': ['doc4', 'doc5', 'doc6']}]
```

In [49]:
import pandas as pd
import numpy as np

In [31]:
golden = pd.DataFrame.from_dict([
    {'query': 'q1', 'document': 'doc2'},
    {'query': 'q1', 'document': 'doc3'},
    {'query': 'q2', 'document': 'doc6'},
])

results = pd.DataFrame.from_dict([
    {'query': 'q1', 'document': 'doc1', 'rank': 1},
    {'query': 'q1', 'document': 'doc2', 'rank': 2},
    {'query': 'q1', 'document': 'doc3', 'rank': 3},
    {'query': 'q2', 'document': 'doc4', 'rank': 1},
    {'query': 'q2', 'document': 'doc5', 'rank': 2},
    {'query': 'q2', 'document': 'doc6', 'rank': 3},
])

In [32]:
golden

,query,document
0,q1,doc2
1,q1,doc3
2,q2,doc6


In [33]:
results

,query,document,rank
0,q1,doc1,1
1,q1,doc2,2
2,q1,doc3,3
3,q2,doc4,1
4,q2,doc5,2
5,q2,doc6,3


In [81]:
def to_nested(results_df):
  res = []
  for q in results_df['query'].unique():
    answers = list(results_df[results_df['query'] == q].sort_values('rank')['document'].values)
    res.append({'question': q, 'answers': answers})
  return res

to_nested(results)

[{'question': 'q1', 'answers': ['doc1', 'doc2', 'doc3']},
 {'question': 'q2', 'answers': ['doc4', 'doc5', 'doc6']}]

In [87]:
def from_nested(results):
  res = []
  for item in results:
    for i, answer in enumerate(item['answers']):
      res.append([item['question'], answer, i+1])
  return pd.DataFrame(res, columns=['query', 'document', 'rank'])

(from_nested(to_nested(results)) == results).all().all()

True

## MRR
https://stackoverflow.com/questions/49733119/calculate-mean-reciprocal-rank

In [101]:
MAX_RANK = 100000

def mrr(golden, results, max_rank=MAX_RANK):
  if isinstance(results, pd.DataFrame):
    res = results
  elif isinstance(results, list):
    res = from_nested(results)
  else:
    raise NotImplementedError()
    
  hits = pd.merge(golden, res,
      on=["query", "document"],
      how="left").fillna(max_rank)

  mrr = (1 / hits.groupby('query')['rank'].min()).mean()
  return mrr

In [102]:
mrr(golden, results)

0.41666666666666663

In [103]:
mrr(golden, to_nested(results))

0.41666666666666663

### Sanity Check

https://gist.github.com/bwhite/3726239

In [104]:
def mean_reciprocal_rank(rs):
    """Score is reciprocal of the rank of the first relevant item
    First element is 'rank 1'.  Relevance is binary (nonzero is relevant).
    Example from http://en.wikipedia.org/wiki/Mean_reciprocal_rank
    >>> rs = [[0, 0, 1], [0, 1, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.61111111111111105
    >>> rs = np.array([[0, 0, 0], [0, 1, 0], [1, 0, 0]])
    >>> mean_reciprocal_rank(rs)
    0.5
    >>> rs = [[0, 0, 0, 1], [1, 0, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.75
    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean reciprocal rank
    """
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])


In [105]:
grouped = hits.groupby('query')['rank'].min().values

rs = np.zeros((len(grouped), grouped.max()))

for i, j in enumerate(grouped):
  rs[i, j-1] = 1

rs

array([[0., 1., 0.],
       [0., 0., 1.]])

In [106]:
mean_reciprocal_rank(rs)

0.41666666666666663

In [107]:
rs = [[0, 0, 0, 1], [1, 0, 0], [1, 0, 0]]
mean_reciprocal_rank(rs)

0.75

In [108]:
rs = [[0, 0, 0, 1], [1, 1, 0], [1, 1, 0]]
mean_reciprocal_rank(rs)

0.75